# Imports

In [ ]:
!pip install --upgrade pyalgotrading

In [ ]:
from pyalgotrading.algobulls import AlgoBullsConnection

# Connection

In [ ]:
connection = AlgoBullsConnection()

In [ ]:
connection.get_token_url()

In [ ]:
API_TOKEN = "c3477e22990b004e542ac6eaee59d3cc961816f5"
connection.set_access_token(API_TOKEN)

#### NOTE: Before creating or executing the strategy, ensure your Alpaca account is binded with your AlgoBulls account (as shown below). To know more click [here](https://algobulls.github.io/pyalgotrading/brokers/connecting_alpaca/).![Reference](https://algobulls.github.io/pyalgotrading/python_build/imgs_v2/python_build_Oauth_successful.png)

# Strategy Creation

## Import Strategy from pyalgostrategypool

In [ ]:
! wget -O obv_breakout_us.py https://raw.githubusercontent.com/algobulls/pyalgostrategypool/master/pyalgostrategypool/obv_breakout/_strategy.py
! sed -i '1s/^/from pyalgotrading.strategy import StrategyBase\n/' obv_breakout_us.py

In [ ]:
from obv_breakout_us import OBVBreakoutStrategy as strategy_cls

In [ ]:
response = connection.create_strategy(strategy_cls, overwrite=True)
response

In [ ]:
strategy = response['strategyId']

# Strategy Testing

## Instruments Searching (optional)

In [ ]:
instrument = connection.search_instrument('MSFT', exchange='NASDAQ')[0]['value']
instrument

## Setup Parameters

In [ ]:
parameters = {
    'OBV_MA_PERIOD': 10,
    'LOOKBACK_PERIOD': 20,
    'TRAILING_STOP_PERIOD': 10,
    'ATR_PERIOD': 14,
    'ATR_MULTIPLIER': 2
}

In [ ]:
initial_virtual_funds = 4000     # in dollars

## Backtesting

#### NOTE: Ensure your Alpaca account is binded with your AlgoBulls account (as shown below). To know more click [here](https://algobulls.github.io/pyalgotrading/brokers/connecting_alpaca/).

### Start

In [ ]:
connection.backtest(
    strategy=strategy, 
    start='2021-08-01 09:15 -0400', 
    end='2023-07-31 15:30 -0400', 
    instrument='NASDAQ:MSFT', 
    lots=5,
    parameters=parameters, 
    candle='1 hour',
    initial_funds_virtual=initial_virtual_funds
)

### Status

In [ ]:
connection.get_backtesting_job_status(strategy)

### Logs

In [ ]:
logs = connection.get_backtesting_logs(strategy)

In [ ]:
print(logs)

### Stop

In [ ]:
connection.stop_backtesting_job(strategy)

## Profit and Loss Reports

In [ ]:
pnl_reports = connection.get_backtesting_report_pnl_table(strategy)
pnl_reports

## Statistics Reports

### Statistics

In [ ]:
connection.get_backtesting_report_statistics(strategy)

### Quantstats Full Report

In [ ]:
connection.get_backtesting_report_statistics(strategy, report='full', html_dump=True)

## Order History

In [ ]:
order_history = connection.get_backtesting_report_order_history(strategy)
print(order_history)

## Papertrading

#### NOTE: Ensure your Alpaca account is binded with your AlgoBulls account (as shown below). To know more click [here](https://algobulls.github.io/pyalgotrading/brokers/connecting_alpaca/).

### Start

In [ ]:
connection.papertrade(
    strategy=strategy, 
    start='9:15 -0400', 
    end='15:00 -0400',  
    instruments='NASDAQ:MSFT', 
    lots=5,
    parameters=parameters, 
    candle='1 minute'
)

### Status

In [ ]:
connection.get_papertrading_job_status(strategy)

### Logs

In [ ]:
logs = connection.get_papertrading_logs(strategy)
print(logs)

### Stop

In [ ]:
connection.stop_papertrading_job(strategy)

### Profit and Loss Reports (Paper Trading)

In [ ]:
pnl_reports = connection.get_papertrading_report_pnl_table(strategy)
pnl_reports

### Statistics Reports (Paper Trading)

#### Statistics

In [ ]:
connection.get_papertrading_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_papertrading_report_statistics(strategy, mode='quantstats', report='full', html_dump=True)

### Order History (Papertrading)

In [ ]:
order_history = connection.get_papertrading_report_order_history(strategy)
print(order_history)

## Live Trading (Real Trading)

#### NOTE: Ensure your Alpaca account is binded with your AlgoBulls account (as shown below). To know more click [here](https://algobulls.github.io/pyalgotrading/brokers/connecting_alpaca/).

### Start

In [ ]:
broking_details = {
    'brokerName': 'ALPACA LIVE',
    'credentialParameters': {}
}

In [ ]:
connection.realtrade(
    strategy=strategy, 
    start='8:00 -0400', 
    end='15:00 -0400', 
    instruments='NASDAQ:MSFT', 
    lots=5,
    parameters=parameters, 
    candle='1 minute',
    broking_details=broking_details
)

### Status

In [ ]:
connection.get_realtrading_job_status(strategy)

### Logs

In [ ]:
logs = connection.get_realtrading_logs(strategy)
print(logs)

### Stop

In [ ]:
connection.stop_realtrading_job(strategy)

### Profit and Loss Reports (Live Trading)

In [ ]:
pnl_reports = connection.get_realtrading_report_pnl_table(strategy)
pnl_reports

### Statistics Reports (Live Trading)

#### Statistics

In [ ]:
connection.get_realtrading_report_statistics(strategy)

#### Quantstats Full Report

In [ ]:
connection.get_realtrading_report_statistics(strategy, mode='quantstats', report='full', html_dump=True)